In [89]:
import pandas as pd
from pandas import read_csv as read
import matplotlib.pyplot as plt
import seaborn as sns

In [90]:
df_original = read('../outputs/general/test_general.csv')
df_classification = read('../classification/outputs/test_general.csv')
df_limited = read('../general_limited/outputs/test_general.csv')

df_original['type'] = "original"
df_original = df_original.rename(columns={"Training Subjects": "test_set"})
df_limited['type'] = "limited"

df = pd.concat([df_original, df_limited])
df = df[df['Model'] != "Least Angle Regression"]

In [91]:
subject_type = []

subjects = set()
for subjects_tuple in df['test_set']:
    subjects.update(eval(subjects_tuple))

for t in ["original", "limited"]:
    for s in subjects:
        subject_type.append(
            (s, t)
        )

In [92]:
# Create the dataframe with an additional 'Type' column in the index
columns = ['Best Model', 'Min MAE', 'Median MAE', 'Max MAE', 'Other Participants']
index = pd.MultiIndex(levels=[[],[]], codes=[[],[]], names=['Subject', 'Type'])
df_summary_general = pd.DataFrame(columns=columns, index=index)

# Loop through each unique combination of subject and type
for (subject, type_) in subject_type:
    # Filter the dataframe for rows where the current subject and type match
    subject_df = df[df['test_set'].apply(lambda x: subject in eval(x))]
    subject_type_df = subject_df[subject_df['type'] == type_]

    # print("S: " + str(subject) + ", T: " + type_)
    # print(subject_type_df.test_set.unique())
    # print(subject_type_df.type.unique())

    if len(subject_type_df.index) != 0:
        # Find the model with the minimum MAE for the current combination of subject and type
        best_model_row = subject_type_df.loc[subject_type_df['MAE'].idxmin()]
        best_model = best_model_row['Model']

        # Find the min MAE
        min_mae = best_model_row['MAE']

        # Find the median MAE
        median_mae = subject_type_df['MAE'].median()

        # Find the max MAE
        max_mae = subject_type_df.loc[subject_type_df['MAE'].idxmax()]['MAE']

        # Find the 'Other Participants' in the best model scenario
        best_training_subjects = eval(best_model_row['test_set'])
        other_participants = tuple(set(best_training_subjects) - {subject})

        # Append to the dataframe for the current subject and type
        df_summary_general.loc[(subject, type_), :] = [best_model, min_mae, median_mae, max_mae, other_participants]

# Sort the index of the dataframe
# df_summary_general.sort_index(inplace=True)

c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return asarray(a).ndim
c:\Users\Build\Documents\VSCode\shoulder_fatigue_modeling\venv\lib\site-packages\numpy\core\fromnumeric.py:3156: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (wh

In [93]:
df_original

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,test_set,type
0,Extra Trees Regressor,2.3303,7.6826,2.7717,-0.0401,0.7545,0.8370,"(1, 2, 3)",original
1,Light Gradient Boosting Machine,2.5592,9.5300,3.0871,-0.2902,0.8679,1.0594,"(1, 2, 3)",original
2,Random Forest Regressor,2.3454,7.7616,2.7860,-0.0508,0.7799,0.8794,"(1, 2, 3)",original
3,Gradient Boosting Regressor,2.5177,9.0438,3.0073,-0.2243,0.8310,0.9756,"(1, 2, 3)",original
4,Linear Regression,2.4823,9.1423,3.0236,-0.2377,0.8478,0.9877,"(1, 2, 3)",original
...,...,...,...,...,...,...,...,...,...
5917,Decision Tree Regressor,2.9949,16.1364,4.0170,-1.7385,1.0703,1.3934,"(8, 16, 17)",original
5918,K Neighbors Regressor,2.1717,7.6279,2.7619,-0.2945,0.7221,1.0156,"(8, 16, 17)",original
5919,Dummy Regressor,1.9922,5.9424,2.4377,-0.0085,0.6691,0.9982,"(8, 16, 17)",original
5920,Passive Aggressive Regressor,2.3340,10.4297,3.2295,-0.7700,0.9151,0.7158,"(8, 16, 17)",original


In [95]:
df_summary_general = df_summary_general.reset_index()
df_summary_general.to_csv('summary_general_comparison.csv', mode='w', header=True, index=False)


In [100]:
df_classification_summary = df_classification.groupby('Model').agg({'Accuracy': 'median', 'Recall': 'median', 'Prec.': 'median', 'F1': 'median'}).reset_index()
df_classification_summary.to_csv('df_classification_summary.csv', mode='w', header=True, index=False)